In [2]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
import pretty_midi
import warnings
import os

In [3]:
def get_genres(path):
    ids = []
    genres = []
    with open(path) as f:
        line = f.readline()
        while line:
            if line[0] != '#':
                [x, y, *_] = line.strip().split("\t")
                ids.append(x)
                genres.append(y)
            line = f.readline()
    genre_df = pd.DataFrame(data={"Genre": genres, "TrackID": ids})
    return genre_df

# Get the Genre DataFrame
genre_path = "msd_tagtraum_cd1.cls"
genre_df = get_genres(genre_path)

# Create Genre List and Dictionary
label_list = list(set(genre_df.Genre))
label_dict = {lbl: label_list.index(lbl) for lbl in label_list}

# Print to Visualize
print(genre_df.head(), end="\n\n")
print(label_list, end="\n\n")
print(label_dict, end="\n\n")

np.save('label_list', label_list)

      Genre             TrackID
0  Pop_Rock  TRAAAAK128F9318786
1       Rap  TRAAAAW128F429D538
2  Pop_Rock  TRAAABD128F429CF47
3      Jazz  TRAAAED128E0783FAB
4  Pop_Rock  TRAAAEF128F4273421

['Vocal', 'International', 'RnB', 'Rap', 'Pop_Rock', 'Country', 'Electronic', 'New Age', 'Reggae', 'Blues', 'Jazz', 'Latin', 'Folk']

{'Vocal': 0, 'International': 1, 'RnB': 2, 'Rap': 3, 'Pop_Rock': 4, 'Country': 5, 'Electronic': 6, 'New Age': 7, 'Reggae': 8, 'Blues': 9, 'Jazz': 10, 'Latin': 11, 'Folk': 12}



In [4]:
def get_matched_midi(midi_folder, genre_df):
    # Get All Midi Files
    track_ids, file_paths = [], []
    for dir_name, subdir_list, file_list in os.walk(midi_folder):
        if len(dir_name) == 36:
            track_id = dir_name[18:]
            file_path_list = ["/".join([dir_name, file]) for file in file_list]
            for file_path in file_path_list:
                track_ids.append(track_id)
                file_paths.append(file_path)
    all_midi_df = pd.DataFrame({"TrackID": track_ids, "Path": file_paths})
    # Inner Join with Genre Dataframe
    df = pd.merge(all_midi_df, genre_df, on='TrackID', how='inner')
    return df.drop(["TrackID"], axis=1)

# Obtain DataFrame with Matched Genres to File Paths
midi_path = "lmd_matched"
matched_midi_df = get_matched_midi(midi_path, genre_df)

# Print to Check Correctness
print(matched_midi_df.head(100))
matched_midi_arr = matched_midi_df.to_numpy()

np.save('matched_midi_arr', matched_midi_arr)

                                                 Path     Genre
0   lmd_matched\A\A\A\TRAAAGR128F425B14B/1d9d16a9d...  Pop_Rock
1   lmd_matched\A\A\A\TRAAAGR128F425B14B/5dd29e99e...  Pop_Rock
2   lmd_matched\A\A\A\TRAAAGR128F425B14B/b97c529ab...  Pop_Rock
3   lmd_matched\A\A\D\TRAADKW128E079503A/3797e9b9a...  Pop_Rock
4   lmd_matched\A\A\D\TRAADKW128E079503A/72cae5077...  Pop_Rock
..                                                ...       ...
95  lmd_matched\A\D\W\TRADWNF128E0792485/d8f369f6d...  Pop_Rock
96  lmd_matched\A\D\Y\TRADYDU128F1484B2C/59eba9233...  Pop_Rock
97  lmd_matched\A\D\Y\TRADYQK128F42A762A/e6190776c...  Pop_Rock
98  lmd_matched\A\D\Z\TRADZQV128F14A5760/02983c512...       RnB
99  lmd_matched\A\D\Z\TRADZQV128F14A5760/507b29065...       RnB

[100 rows x 2 columns]


In [4]:
%%time
def normalize_features(features):
    #This function normalizes the features to the range [-1, 1]
    tempo = (features[0] - 150) / 300
    num_sig_changes = (features[1] - 2) / 10
    resolution = (features[2] - 260) / 400
    time_sig_1 = (features[3] - 3) / 8
    time_sig_2 = (features[4] - 3) / 8
    return [tempo, resolution, time_sig_1, time_sig_2]


def get_features(path):
    '''
    try:
        # Test for Corrupted Midi Files
        with warnings.catch_warnings():
            warnings.simplefilter("error")
            file = pretty_midi.PrettyMIDI(path)
            
            tempo = file.estimate_tempo()
            num_sig_changes = len(file.time_signature_changes)
            resolution = file.resolution
            ts_changes = file.time_signature_changes
            ts_1 = 4
            ts_2 = 4
            if len(ts_changes) > 0:
                ts_1 = ts_changes[0].numerator
                ts_2 = ts_changes[0].denominator
            return normalize_features([tempo, num_sig_changes, resolution, ts_1, ts_2])
    except:
        return None
    '''
    print(path)
    file = pretty_midi.PrettyMIDI(path)
            
    tempo = file.estimate_tempo()
    num_sig_changes = len(file.time_signature_changes)
    resolution = file.resolution
    ts_changes = file.time_signature_changes
    ts_1 = 4
    ts_2 = 4
    if len(ts_changes) > 0:
        ts_1 = ts_changes[0].numerator
        ts_2 = ts_changes[0].denominator
    return normalize_features([tempo, num_sig_changes, resolution, ts_1, ts_2])


def extract_midi_features(path_df):
    all_features = []
    for index, row in path_df.iterrows():
        features = get_features(row.Path)
        genre = label_dict[row.Genre]
        if features is not None:
            features.append(genre)
            all_features.append(features)
    return np.array(all_features)

labeled_features = extract_midi_features(matched_midi_df)
print(labeled_features)

#it will run for almost 1 hour QQ
np.save('labeled_features', labeled_features)

lmd_matched\A\A\A\TRAAAGR128F425B14B/1d9d16a9da90c090809c153754823c2b.mid
lmd_matched\A\A\A\TRAAAGR128F425B14B/5dd29e99ed7bd3cc0c5177a6e9de22ea.mid
lmd_matched\A\A\A\TRAAAGR128F425B14B/b97c529ab9ef783a849b896816001748.mid
lmd_matched\A\A\D\TRAADKW128E079503A/3797e9b9a101c110b7375c74dd4c70b1.mid
lmd_matched\A\A\D\TRAADKW128E079503A/72cae5077339f6abaee4cad318b1e923.mid
lmd_matched\A\A\F\TRAAFMT128F429DB58/0a4f2051b572ebe007647fcab0114dd8.mid
lmd_matched\A\A\G\TRAAGMC128F4292D0F/0644195d1a3d14e0a0bd3d8b30dc68da.mid
lmd_matched\A\A\G\TRAAGMC128F4292D0F/1fecb8a2e5e9df4df423793909d52d33.mid
lmd_matched\A\A\G\TRAAGMC128F4292D0F/3bcef98aa322c4a083efe43a962768d8.mid
lmd_matched\A\A\G\TRAAGMC128F4292D0F/52b6febfe8da6bb7c2653aa7e982297f.mid
lmd_matched\A\A\G\TRAAGMC128F4292D0F/7c2e695d0593d35ca3b0913549754f18.mid
lmd_matched\A\A\L\TRAALAH128E078234A/8cfecf566c75342d13c327f8a7abc414.mid
lmd_matched\A\A\L\TRAALAH128E078234A/b3e2c3f566c5e20de4cd4e3c262addad.mid
lmd_matched\A\A\N\TRAANTZ128E0792529/6

C:\Users\user\anaconda3\envs\midienv\lib\site-packages\pretty_midi\pretty_midi.py:97: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


lmd_matched\A\B\V\TRABVVN12903CB6445/1dbcd2e2ba968cbd1f9cab03a81f2adb.mid
lmd_matched\A\C\D\TRACDHB12903CDF8F6/d5d063749eb300431d7caec2b039dccc.mid
lmd_matched\A\C\D\TRACDLP12903CBE918/eeeea7ca7185792d5a0a4a9ef85d7838.mid
lmd_matched\A\C\H\TRACHRV128F4214FB6/020ae8f827b69c9e6f953284f75542c6.mid
lmd_matched\A\C\H\TRACHRV128F4214FB6/5dd4427d17c9f72d41231ac40337f30b.mid
lmd_matched\A\C\J\TRACJWL128F92FB60E/31a3a52f9d682d9621d8cae6303aad72.mid
lmd_matched\A\C\J\TRACJWL128F92FB60E/b242fbf1d8837bac7570e84c0813aeff.mid
lmd_matched\A\C\J\TRACJWL128F92FB60E/f06251373c71541ff02a7bf6375b4d88.mid
lmd_matched\A\C\K\TRACKAP128F425B741/202e8bf3824576d0a3925a4bda048b65.mid
lmd_matched\A\C\K\TRACKAP128F425B741/5d75b6e38a570be3a95ea11c7b2bdfe7.mid
lmd_matched\A\C\M\TRACMMC128F92FA031/1482d282c46edf585b9ee727776d482d.mid
lmd_matched\A\C\M\TRACMMC128F92FA031/548e89688846fd4f1bbfc8209f8a3748.mid
lmd_matched\A\C\N\TRACNMH12903CA9CCC/c1f6817c7e5a03aa5214ff1db0b7d7d8.mid
lmd_matched\A\C\O\TRACOVU128EF34B7F4/b

lmd_matched\A\F\M\TRAFMVN128F147CBCE/ad126ea6c63c8eb5b1397e2aba8a5f50.mid
lmd_matched\A\F\M\TRAFMVN128F147CBCE/e9b281b4d9434eb58c40f5ec29168123.mid
lmd_matched\A\F\Q\TRAFQFM128E078EC97/3d5a78da77f009eb88641731e91d6982.mid
lmd_matched\A\F\U\TRAFUHN128F42BC199/7065a08391ee86cbbd393ba94b732739.mid
lmd_matched\A\F\X\TRAFXZH128F4262D0A/39a063283db8902fab12e5b57c2fc285.mid
lmd_matched\A\F\X\TRAFXZH128F4262D0A/de458b3b942f6c7b9bf5783b81afcba5.mid
lmd_matched\A\F\Y\TRAFYGA128F425A92C/1d8d97a379a1e07161a55727f6ee311e.mid
lmd_matched\A\F\Y\TRAFYGA128F425A92C/307ee52d20b05c234fa65c79ce1f2f3e.mid
lmd_matched\A\F\Y\TRAFYGA128F425A92C/58778d272f64b8a9df8a5f735d3a64a3.mid
lmd_matched\A\F\Y\TRAFYGA128F425A92C/da40de891b0e766ee370da0246127290.mid
lmd_matched\A\F\Y\TRAFYGA128F425A92C/f19d3b6e19ace71ae3ba6a1c4d422f58.mid
lmd_matched\A\F\Z\TRAFZAM128F426E915/5693bace21c164e8508b124b7faa9f3d.mid
lmd_matched\A\F\Z\TRAFZAM128F426E915/ac487967d8b6d5420553ae13d4ecea0d.mid
lmd_matched\A\G\E\TRAGETL128F1492172/2

lmd_matched\A\J\K\TRAJKQZ128F1465490/d0c2648c70385204360c2a427860419b.mid
lmd_matched\A\J\K\TRAJKQZ128F1465490/d212dfd1239893af995c68ae680df41a.mid
lmd_matched\A\J\K\TRAJKQZ128F1465490/d228ac01cfaea70ff31e479908029dca.mid
lmd_matched\A\J\K\TRAJKQZ128F1465490/d6d3c62f526258332a4ba2f560e1b598.mid
lmd_matched\A\J\K\TRAJKQZ128F1465490/d8113519569d9f91c7f8e7127c6fecbc.mid
lmd_matched\A\J\K\TRAJKQZ128F1465490/dbb38ce2da133b97eff7025c2f9ed658.mid
lmd_matched\A\J\K\TRAJKQZ128F1465490/e3a1f1d5be7496086d318a01b490c167.mid
lmd_matched\A\J\N\TRAJNYU128E07856A5/e9c0888906be0751b408500a831f333c.mid
lmd_matched\A\J\Z\TRAJZLQ128F424E12B/41f8d03a1995e82dd8fc8bfb5da32e24.mid
lmd_matched\A\K\D\TRAKDIG128E07826B3/0245ac9a4a550e3f2a472c6a5853e9ea.mid
lmd_matched\A\K\D\TRAKDIG128E07826B3/50e79c8b4ee6a06a2661f1661cb51ded.mid
lmd_matched\A\K\D\TRAKDIG128E07826B3/5db219e65c684d3b4f5ebc02bf4580a8.mid
lmd_matched\A\K\D\TRAKDIG128E07826B3/8a494c8853f504c20ce863ea5f95ad89.mid
lmd_matched\A\K\D\TRAKDIG128E07826B3/d

...

lmd_matched\Y\W\H\TRYWHWI128F145981E/c7490fa02012469c7db22eeaebaa5cf2.mid
lmd_matched\Y\W\H\TRYWHWI128F145981E/f5f0ec1a476ae7242a4ac0ec64e0084c.mid
lmd_matched\Y\W\J\TRYWJBC128F92FA09B/d665b72ccfc90d4a93ebdedd47b1f475.mid
lmd_matched\Y\W\M\TRYWMYW12903D139C3/0c50ef3276d2cb78ae1cc5e2e7bdfd95.mid
lmd_matched\Y\W\M\TRYWMYW12903D139C3/15e4fb1dd06ae3b98540025813597ab8.mid
lmd_matched\Y\W\M\TRYWMYW12903D139C3/2dfc2e66f4cace81c83f18e2d2f12f74.mid
lmd_matched\Y\W\M\TRYWMYW12903D139C3/339d1460748dd3566e883bb77f6e9200.mid
lmd_matched\Y\W\M\TRYWMYW12903D139C3/37fa27f23579ad9c3ffb66f0cbebc41b.mid
lmd_matched\Y\W\M\TRYWMYW12903D139C3/390c984f4cd4cae0cca3c31fff304169.mid
lmd_matched\Y\W\M\TRYWMYW12903D139C3/61c9db58a49b5df601d426caa07f5994.mid
lmd_matched\Y\W\M\TRYWMYW12903D139C3/7fc20a5bb49e0207463bf339db1b841c.mid
lmd_matched\Y\W\M\TRYWMYW12903D139C3/95b4f0e48ae8c9878755548403ddc3a7.mid
lmd_matched\Y\W\M\TRYWMYW12903D139C3/a470e7898eee2addfadc0ee3461cc109.mid
lmd_matched\Y\W\M\TRYWMYW12903D139C3/a

lmd_matched\Z\D\S\TRZDSIO128F42868A8/6c17e71735158a52a7f50c0bc3b7c978.mid
lmd_matched\Z\D\S\TRZDSIO128F42868A8/6f991d772197651eb6667a12a5425041.mid
lmd_matched\Z\D\S\TRZDSIO128F42868A8/70806d53054172c121f8d9edc2b4cbef.mid
lmd_matched\Z\D\S\TRZDSIO128F42868A8/73bd81048a93cad7c643b546a2df2304.mid
lmd_matched\Z\D\S\TRZDSIO128F42868A8/749e5d024eaae7a09601ee719545677b.mid
lmd_matched\Z\D\S\TRZDSIO128F42868A8/74f597325e745f17d6ca198aea21e589.mid
lmd_matched\Z\D\S\TRZDSIO128F42868A8/74f6f30bd51ece3e62bcf009fd1ef6fa.mid
lmd_matched\Z\D\S\TRZDSIO128F42868A8/76651e6d1f4cf3e30c6b76f21031b0c3.mid
lmd_matched\Z\D\S\TRZDSIO128F42868A8/77bb2150771a0936b249cc29fa6ac66c.mid
lmd_matched\Z\D\S\TRZDSIO128F42868A8/783ee2aacbc238b3ee58a1f210a6c09d.mid
lmd_matched\Z\D\S\TRZDSIO128F42868A8/788f58be65b9099db070a88952c74a8a.mid
lmd_matched\Z\D\S\TRZDSIO128F42868A8/7ba91a37966de806e94187ee10f0d2a8.mid
lmd_matched\Z\D\S\TRZDSIO128F42868A8/7eceee30979968a9b10ced13b42c93df.mid
lmd_matched\Z\D\S\TRZDSIO128F42868A8/7

lmd_matched\Z\G\Z\TRZGZOF128F424E305/1170f8a25713b9a2e3dc2a9ba245835f.mid
lmd_matched\Z\G\Z\TRZGZOF128F424E305/91c8c9d8a944035228f73eb2ec8ff2ff.mid
lmd_matched\Z\G\Z\TRZGZOF128F424E305/aea0ffdbb75bd3c6058f089f3b6300ab.mid
lmd_matched\Z\G\Z\TRZGZOF128F424E305/bbad2e429e32bb5b038f1e18adf92675.mid
lmd_matched\Z\G\Z\TRZGZOF128F424E305/c2f1a3f0b400702e68ff08b252c962e3.mid
lmd_matched\Z\G\Z\TRZGZOF128F424E305/c550dccd09392420ea6705b2b613013a.mid
lmd_matched\Z\G\Z\TRZGZOF128F424E305/c7c495a179aa93a0357ae07db60eb01d.mid
lmd_matched\Z\H\N\TRZHNBD128F14AA6AD/539d9440b9b6f2527e1c1512d92e4f59.mid
lmd_matched\Z\H\Z\TRZHZBC128F14750BF/19c1fba8009229e82ff8f06b45795925.mid
lmd_matched\Z\H\Z\TRZHZBC128F14750BF/6659bc7887d9f3249c1b677b265109a5.mid
lmd_matched\Z\H\Z\TRZHZBC128F14750BF/89ec6303ba05f58e00644b6ee36140f8.mid
lmd_matched\Z\H\Z\TRZHZBC128F14750BF/af916982865e14b7af5d630e1912d88a.mid
lmd_matched\Z\H\Z\TRZHZBC128F14750BF/d9e5ec6f76d8a78b68eca186c2baf32a.mid
lmd_matched\Z\I\D\TRZIDFM128F9306A05/9

lmd_matched\Z\P\G\TRZPGNC128F42A2230/a9539b8fcfab9c7ea70d3ff33959ca53.mid
lmd_matched\Z\P\G\TRZPGNC128F42A2230/c35393c4996bb3ac92e586463c202009.mid
lmd_matched\Z\P\V\TRZPVLJ128F148D2F7/221fee97d8075c31814affb58bd2ed29.mid
lmd_matched\Z\P\V\TRZPVLJ128F148D2F7/c0cbb104500c782d3bbf42964defa1c4.mid
lmd_matched\Z\P\V\TRZPVLJ128F148D2F7/e00cf027e60fffe5f43ee6e17c565f11.mid
lmd_matched\Z\P\V\TRZPVLJ128F148D2F7/eddcf051d86783139ed040e26b83ebef.mid
lmd_matched\Z\P\V\TRZPVLJ128F148D2F7/f3d9a2a0f3775884a6221c59ceb2c897.mid
lmd_matched\Z\P\Y\TRZPYZQ128F1494D71/b68ff32e4d48d5c8821012e9ad2c9d65.mid
lmd_matched\Z\Q\H\TRZQHOP128F427EE59/bb039365d12fb825c72b1b9f156cfc2d.mid
lmd_matched\Z\Q\M\TRZQMUQ128EF345494/b43b4c1ff299e824fe1a92fbab4c5517.mid
lmd_matched\Z\Q\Q\TRZQQLP128F92DD94E/0205c9a243198de07cb80149422d7e20.mid
lmd_matched\Z\Q\Q\TRZQQLP128F92DD94E/0b9d48638d37edc8b4e6ee06dbf36b4d.mid
lmd_matched\Z\Q\Q\TRZQQLP128F92DD94E/1645fe4e23b54de9d8a08166641355c5.mid
lmd_matched\Z\Q\Q\TRZQQLP128F92DD94E/1

lmd_matched\Z\U\R\TRZURPG128F92F2ACB/d2def713a43bca4e95a4cbf502d7b6af.mid
lmd_matched\Z\U\R\TRZURPG128F92F2ACB/ee52adde812ebcca0b1acff505b5a4d2.mid
lmd_matched\Z\U\U\TRZUUSY128F931F540/d9034bb3e7bffd7d722f79f12ff95186.mid
lmd_matched\Z\U\X\TRZUXGH128F428EA1B/63cc3cb1e97a72d84daba350cf942c59.mid
lmd_matched\Z\V\B\TRZVBSY128F424AE4E/69677dab444e406287ee1d1dfc86d3f7.mid
lmd_matched\Z\V\D\TRZVDCV12903CBB501/071d8bc9bd8ce7a8264480c077159d82.mid
lmd_matched\Z\V\D\TRZVDEG128F93010FD/7246ec038e33530b5d0e2d5a7bfbe111.mid
lmd_matched\Z\V\P\TRZVPZQ128E078F0E3/38a4712f2efc1802cdbd7a39730156b4.mid
lmd_matched\Z\V\P\TRZVPZQ128E078F0E3/5c398afdfbb4e18ac52b5e3c96fbeb00.mid
lmd_matched\Z\V\P\TRZVPZQ128E078F0E3/df28c7b9c5e745781b5d0d4868af5a73.mid
lmd_matched\Z\V\P\TRZVPZQ128E078F0E3/ed796033ae4ba0ab248923b339b28466.mid
lmd_matched\Z\W\W\TRZWWSB128F4291A86/a0604c7d1a302c393eb42a1682acf9d4.mid
lmd_matched\Z\X\C\TRZXCAI12903CDF52F/392b132f5a24410123fbc6b7dd727861.mid
lmd_matched\Z\X\H\TRZXHUT128F92D8744/a